In [1]:
#from google.colab import drive
#drive.mount("/content/drive/")

IMPORTING PACKAGES

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Convolution2D,MaxPooling2D,Flatten
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.applications import VGG16
import cv2
from tensorflow.keras.models import load_model
import numpy as np


IMPORTING TRAINING DATA

In [2]:
image_generator=ImageDataGenerator(vertical_flip=False,horizontal_flip=True,shear_range=0.1,zoom_range=0.1,rescale=1/255,brightness_range=(0.2,0.8))
X_train1=image_generator.flow_from_directory(target_size=(224,224),
                                    directory="C:\\Users\\ASUS\\Desktop\\AI COURSE\\Project\\training1",
                                    class_mode="categorical",
                                   batch_size=10,
                                            subset="training")

IMPORTING TESTING DATA

In [4]:
image_generator_1=ImageDataGenerator(vertical_flip=False,horizontal_flip=True,shear_range=0.1,zoom_range=0.1,rescale=1/255,brightness_range=(0.2,0.8))
X_test1=image_generator_1.flow_from_directory(target_size=(224,224),
                                    directory="C:\\Users\\ASUS\\Desktop\\AI COURSE\\Project\\validation1",
                                    class_mode="categorical",
                                   batch_size=10,
                                )

Found 171 images belonging to 3 classes.


INITIALIZING MODEL

In [6]:
vgg16=VGG16(include_top=False,input_shape=(224,224,3),weights='imagenet')
for i in vgg16.layers:
  i.trainable=False

ADD FLATTEN LAYER

In [ ]:
flatten_layer=Flatten()(vgg16.output)

ADDING DENSE LAYER

In [ ]:
dense32=Dense(32,kernel_initializer=RandomNormal,activation="relu")(flatten_layer)
output=Dense(3,activation="softmax")(dense32)

BUILDING MODEL

In [22]:
model1=Model(inputs=vgg16.input,outputs=output)
model1.summary()

INITILAIZE LEARNING PARAMETERS

In [23]:
model1.compile(loss=CategoricalCrossentropy(),
              optimizer=Adam(epsilon=0.001),
              metrics=["acc"])

FITTING DATA TO THE MODEL

In [ ]:
model1.fit(X_train1,validation_data=X_test1,epochs=5,steps_per_epoch=30,validation_batch_size=30)

SAVING THE MODEL

In [45]:
model1.save("LevelModel.h5")

LOAD MODEL

In [ ]:
model = load_model('LevelModel.h5')

In [ ]:
def detect(frame):
    img = cv2.resize(frame, (224, 224))
    if(np.max(img) > 1):
        img = img/255.0
    img = np.array([img])
    prediction = model.predict(img)
    label = ["minor", "moderate", "severe"]
    preds = label[np.argmax(prediction)]
    return preds